# 앙상블 검색기(Ensemble Retriever)

`EnsembleRetriever`는 여러 검색기를 결합하여 더 강력한 검색 결과를 제공하는 LangChain의 기능입니다. 이 검색기는 다양한 검색 알고리즘의 장점을 활용하여 단일 알고리즘보다 더 나은 성능을 달성할 수 있습니다.

**주요 특징**
1. 여러 검색기 통합: 다양한 유형의 검색기를 입력으로 받아 결과를 결합합니다.
2. 결과 재순위화: [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) 알고리즘을 사용하여 결과의 순위를 조정합니다.
3. 하이브리드 검색: 주로 `sparse retriever`(예: BM25)와 `dense retriever`(예: 임베딩 유사도)를 결합하여 사용합니다.

**장점**
- Sparse retriever: **키워드 기반 검색**에 효과적
- Dense retriever: **의미적 유사성 기반 검색**에 효과적

이러한 상호 보완적인 특성으로 인해 `EnsembleRetriever`는 다양한 검색 시나리오에서 향상된 성능을 제공할 수 있습니다.

자세한 내용은 [LangChain 공식 문서](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble)를 참조하세요.


In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH10-Retriever")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH10-Retriever


- `EnsembleRetriever`를 초기화하여 `BM25Retriever`와 `FAISS` 검색기를 결합합니다. 각 검색기의 가중치를 설정됩니다.


### BM25Retriever

BM25는 **'Best Match 25'**의 약자로, 검색 엔진에서 문서와 검색어의 **관련성(relevance)**을 점수화하여 순위를 매기는 데 사용되는 알고리즘입니다.

문서 내에 특정 키워드가 얼마나 자주 등장하는지, 그리고 그 키워드가 전체 문서들 중에서 얼마나 희귀한지를 계산하는 키워드 기반의 검색 알고리즘입니다.

BM25의 핵심 원리 🧐
BM25는 TF-IDF(단어 빈도-역 문서 빈도) 개념을 개선하여 다음과 같은 세 가지 주요 요소를 고려합니다.

1. 단어 빈도 (Term Frequency): 문서에 특정 단어가 많이 나올수록 그 문서는 더 관련성이 높다고 판단합니다. 하지만 너무 많이 반복되는 단어는 점수를 낮추는 '포화(saturation)' 개념을 적용해 과도한 점수 부여를 막습니다.

2. 역 문서 빈도 (Inverse Document Frequency): '이', '그', '는'과 같이 모든 문서에 흔하게 등장하는 단어는 중요도가 낮다고 판단합니다. 반면, '양자 컴퓨터', '인공지능'처럼 특정 문서에만 드물게 등장하는 단어는 중요도를 높게 평가합니다.

3. 문서 길이 (Document Length): 문서의 길이가 길수록 특정 단어의 빈도가 높아질 수 있습니다. BM25는 이런 점을 보정하여, 짧은 문서가 동일한 빈도의 단어를 포함할 때 더 높은 점수를 받도록 설계되었습니다.

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 샘플 문서 리스트
doc_list = [                # 예제를 바꿔보기.  # 각 문장은 청크 단위로 분할됨.
    "I like apples",
    "I like apple company",
    "I like apple's iphone",
    "Apple is my favorite company",
    "I like apple's ipad",
    "I like apple's macbook",
]


# bm25 retriever와 faiss retriever를 초기화합니다.
bm25_retriever = BM25Retriever.from_texts(
    doc_list,
)
bm25_retriever.k = 1  # BM25Retriever의 검색 결과 개수를 1로 설정합니다.

embedding = OpenAIEmbeddings()  # OpenAI 임베딩을 사용합니다.
faiss_vectorstore = FAISS.from_texts(
    doc_list,
    embedding,
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 1})

# 앙상블 retriever를 초기화합니다.
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.7, 0.3], # 리트리버의 가중치를 설정. 비율의 합은 1.0이 되도록 설정해야함.
)

`ensemble_retriever` 객체의 `get_relevant_documents()` 메서드를 호출하여 관련성 높은 문서를 검색합니다.


In [ ]:
# 검색 결과 문서를 가져옵니다.
query = "my favorite fruit is apple"
ensemble_result = ensemble_retriever.invoke(query)
bm25_result = bm25_retriever.invoke(query)
faiss_result = faiss_retriever.invoke(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]")
for doc in ensemble_result:
    print(f"Content: {doc.page_content}")
    print()

print("[BM25 Retriever]")
for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()

print("[FAISS Retriever]")
for doc in faiss_result:
    print(f"Content: {doc.page_content}")
    print()

[Ensemble Retriever]
Content: Apple is my favorite company

Content: I like apples

[BM25 Retriever]
Content: Apple is my favorite company

[FAISS Retriever]
Content: I like apples



In [5]:
# 검색 결과 문서를 가져옵니다.
query = "Apple company makes my favorite iphone"
ensemble_result = ensemble_retriever.invoke(query)
bm25_result = bm25_retriever.invoke(query)
faiss_result = faiss_retriever.invoke(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]")
for doc in ensemble_result:
    print(f"Content: {doc.page_content}")
    print()

print("[BM25 Retriever]")
for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()

print("[FAISS Retriever]")
for doc in faiss_result:
    print(f"Content: {doc.page_content}")
    print()

[Ensemble Retriever]
Content: Apple is my favorite company

Content: I like apple's iphone

[BM25 Retriever]
Content: Apple is my favorite company

[FAISS Retriever]
Content: I like apple's iphone



## 런타임 Config 변경

런타임에서도 retriever 의 속성을 변경할 수 있습니다. 이는 `ConfigurableField` 클래스를 사용하여 가능합니다.

- `weights` 매개변수를 `ConfigurableField` 객체로 정의합니다.
  - 필드의 ID는 "ensemble_weights"로 설정합니다.


In [6]:
from langchain_core.runnables import ConfigurableField


ensemble_retriever = EnsembleRetriever(
    # 리트리버 목록을 설정합니다. 여기서는 bm25_retriever와 faiss_retriever를 사용합니다.
    retrievers=[bm25_retriever, faiss_retriever],
).configurable_fields(
    weights=ConfigurableField(
        # 검색 매개변수의 고유 식별자를 설정합니다.
        id="ensemble_weights",
        # 검색 매개변수의 이름을 설정합니다.
        name="Ensemble Weights",
        # 검색 매개변수에 대한 설명을 작성합니다.
        description="Ensemble Weights",
    )
)

- 검색 시 `config` 매개변수를 통해 검색 설정을 지정합니다.
  - `ensemble_weights` 옵션의 가중치를 [1, 0]으로 설정하여 **모든 검색 결과의 가중치가 BM25 retriever 에 더 많이 부여** 되도록 합니다.

In [7]:
config = {"configurable": {"ensemble_weights": [1, 0]}} # 앙상블 가중치를 1:0으로 설정-> 키워드 중심으로 검색.

# config 매개변수를 사용하여 검색 설정을 지정합니다.
docs = ensemble_retriever.invoke("my favorite fruit is apple", config=config)
docs  # 검색 결과인 docs를 출력합니다.

[Document(metadata={}, page_content='Apple is my favorite company'),
 Document(id='a3286142-766f-4bf6-b4d3-989d6fb2e3ff', metadata={}, page_content='I like apples')]

이번에는 검색시 모든 검색 결과의 가중치가 **FAISS retriever 에 더 많이 부여** 되도록 합니다.

In [ ]:
config = {"configurable": {"ensemble_weights": [0, 1]}} # 의미 중심으로 검색.

# config 매개변수를 사용하여 검색 설정을 지정합니다.
docs = ensemble_retriever.invoke("my favorite fruit is apple", config=config)
docs  # 검색 결과인 docs를 출력합니다.

[Document(id='a3286142-766f-4bf6-b4d3-989d6fb2e3ff', metadata={}, page_content='I like apples'),
 Document(metadata={}, page_content='Apple is my favorite company')]